# NA substitution
Fill missing intensities with some values and drop peaks with too many NA
#TODO: Should be replaced with better method

In [1]:
import numpy as np
import pandas as pd

In [2]:
%run 2_columns_part.ipynb

In [139]:
def remove_na_peaks(df, fraction=1):
    """
    Remove peaks which have no values in samples and qc. Need predefined constant samples_wo_controls_qc
    :param df: df - dataframe to clean
    :param fraction: float - threshold of tolerable NA portion for peak
    :return: df - cleaned from empty peaks dataframe
    """
    # Find peaks which contain NA more than provided fraction in samples and qc
    too_many_na = df[samples_wo_controls_qc].isna().sum(axis=1) / df[samples_wo_controls_qc].shape[1] > fraction
    na_peaks = too_many_na[too_many_na].index

    # Get rid of these peaks
    df = df.drop(na_peaks)
    return df

In [140]:
def substitute_na_min(df):
    """
    Substitute NA with values. Need predefined constant samples_wo_controls_qc
    :param df: df - dataframe to clean
    :return: df - dataframe with values in sample area
    """
    df = df.copy()
    
    # Fill NA with 0.5 * min(sample) in rows with intensities
    df[samples_wo_controls_qc] = df[samples_wo_controls_qc].apply(lambda xs: xs.fillna(xs.min() / 2), axis=1)
    return df

There is a 796 peaks which had NA in all samples except controls. I'll drop them.

In [142]:
df[samples_wo_controls_qc].isna().sum().sum()

506422

In [143]:
df = remove_na_peaks(df, 0.5)
print(df[samples_wo_controls_qc].isna().sum().sum(), df[samples_wo_controls_qc].isna().all(axis=1).sum())
df = substitute_na(df)
print(df[samples_wo_controls_qc].isna().sum().sum(), df[samples_wo_controls_qc].isna().all(axis=1).sum())

49487 0
0 0


In [145]:
df.shape

(4233, 142)

In [141]:
df[samples_wo_controls_qc].isna().all(axis=1).sum()

796

## Example of usage with writing substituted dataset

In [110]:
# Load data
name = 'cleaned_control_cleaned_isotopes_cleaned_contaminants_xs_annotated_rats_neg.csv'
df = pd.read_csv(name, index_col=0)

/home/arleg/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (140) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [103]:
# Remove fully NA peaks
df = remove_na_peaks(df)
# Substitute NA
df = substitute_na(df)

# Write to a file
df.to_csv(f'substituted_NA_{name}')